# SemEval 2026 Task 8 - Task C: RAG (End-to-End)

This notebook focuses on **Task C: Retrieval-Augmented Generation**. 
Goal: Given a question, retrieve documents AND generate an answer.

**Output Format**:
The submission file must contain BOTH:
- `contexts`: List of objects (`document_id`, `score`, `text`).
- `predictions`: List of objects (`text` for output answer).

In [ ]:
import os
import json
import sys
from tqdm import tqdm

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    from src.graph import app
except ImportError:
    print("ERROR: Check imports.")

In [ ]:
# --- CONFIGURATION ---
TEAM_NAME = "Gbgers"
TASK_TYPE = "TaskC"
INPUT_FILE = "../dataset/human/generation_tasks/reference.jsonl"
OUTPUT_FILE = f"../data/submissions/submission_{TASK_TYPE}_{TEAM_NAME}.jsonl"

os.makedirs("../data/submissions", exist_ok=True)
print(f"Target: {OUTPUT_FILE}")

In [ ]:
with open(INPUT_FILE) as f:
    test_data = [json.loads(line) for line in f if line.strip()]

results = []
print(f"Running RAG for {len(test_data)} items...")

for item in tqdm(test_data):
    question = item.get("question")
    
    try:
        resp = app.invoke({"question": question})
        raw_docs = resp.get("documents", [])
        gen_text = resp.get("generation", "")
    except Exception as e:
        print(f"Error: {e}")
        raw_docs = []
        gen_text = "Error"
    
    # Format Contexts
    contexts = []
    for doc in raw_docs:
        meta = getattr(doc, "metadata", {})
        contexts.append({
            "document_id": meta.get("id", meta.get("document_id", "unknown")),
            "text": getattr(doc, "page_content", ""),
            "score": float(meta.get("relevance_score", meta.get("score", 0.0)))
        })
        
    # Format Prediction
    predictions = [{"text": gen_text if gen_text else "No Answer"}]
    
    output_item = item.copy()
    output_item["contexts"] = contexts
    output_item["predictions"] = predictions
    results.append(output_item)

print("RAG Execution Completed.")

In [ ]:
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for x in results:
        json.dump(x, f)
        f.write('\n')
print(f"Saved to {OUTPUT_FILE}")

# Validation
sample = results[0]
valid_c = ("contexts" in sample and isinstance(sample["contexts"], list)) and \
          ("predictions" in sample and isinstance(sample["predictions"], list))

if valid_c:
    print("\033[92mVALIDATION PASS: Structure correct for Task C.\033[0m")
else:
    print("\033[91mVALIDATION FAIL: Missing fields.\033[0m")